In [18]:
%load_ext autoreload
%autoreload 2

import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import json
import os
from annual_profit_optimisation import run_profit_maximisation

# Set ERCOT load zone and solar capacity
ERCOT_LOAD_ZONE = 'LZ_HOUSTON'
SOLAR_CAPACITY = 300
SOLAR_PPA_DOL_KWH = 250
FACILITY_LIFETIME = 25

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
# READ IN DATA
# TODO: update as 'electricity' here is a normalised 8760 solar profile
hourly_dayahead_realtime_solar = pd.read_csv("hourly_dayahead_realtime_solar.csv")

# Filter by load zone and complete data
ERCOT_LOAD_ZONE = 'LZ_HOUSTON'
hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar[hourly_dayahead_realtime_solar['zone'] == ERCOT_LOAD_ZONE]
hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar_lz[hourly_dayahead_realtime_solar_lz['electricity'].notna()]

number_of_days_test = 100
hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar_lz.iloc[0:24*number_of_days_test,]
# Sets time duration of the optimisation
# TODO: generat appropriate 8760 - due to shift in UTC time used for solar profile 
# and local_time used for ERCOT realtime and dayahead prices
T = hourly_dayahead_realtime_solar_lz.shape[0]
print(T)

2400


In [50]:
# FACILITY
FACILITY_LIFETIME = 25
OFFTAKE_TPD = 10
GREEN_THRESHOLD_kg_per_kg = 0.4
NUM_DAYS = math.ceil(T/24)
H2_SALES_PRICE_DOL_per_kg = 4.0

# POWER
#   SOLAR
SOLAR_CAPACITY = 400
SOLAR_PPA_DOL_KWH = 0.032 # 2.5 cents per kWh ($USD 50 per MWh)
hourly_solar_production_kwh = hourly_dayahead_realtime_solar_lz['electricity']/hourly_dayahead_realtime_solar_lz['electricity'].max()
#   GRID 
#       CONVERT $/MWh -> $/kWh
hourly_dayahead_dol_per_kwh = hourly_dayahead_realtime_solar_lz['price_dayahead'] / 1000
ERCTO_CO2_kg_per_kwh = 0.42062
hourly_dayahead_dol_per_kwh =  hourly_dayahead_dol_per_kwh.reset_index(drop=True)
# hourly_dayahead_dol_per_kwh.iloc[20] = 0.3

# hourly_dayahead_dol_per_kwh

# EQUIPMENT
#   OPERATIONAL 
#       EFFICIENCY
ELECTROLYSIS_EFFICIENCY = 55
COMPRESSION_EFFICIENCY = 4
LIQUEFACTION_EFFICIENCY = 10
FUEL_CELL_EFFICIENCY = 20
#       LOWER OPERATION BOUNDS (% of nameplate capacity)
ELECTROLYSIS_OPERATION_LOWER = 0.15
COMPRESSOR_OPERATION_LOWER = 0.15
LIQUEFACTION_OPERATION_LOWER = 0.60 
#       RAMP CONSTRAINTS (% of nameplate capacity)
LIQUEFACTION_MAX_UP = 0.2 # 10% per hour up
LIQUEFACTION_MAX_DOWN = 0.2 # 10% per hour down

#   CAPEX
#   CAPEX - total
ELECTROLYSIS_CAPEX_DOL_TPD = 1900000 # 1.9 million $USD per TPD of electrolysis
COMPRESSOR_CAPEX_DOL_TPD = 900000 # 0.8 million $USD per TPD of comrpessor
LIQUEFACTION_CAPEX_DOL_TPD = 2945000 # 2.945 million $USD per TPD of liquefaction per day
FUELCELL_CAPEX_DOL_TPD = 100000 # 1.945 million $USD per TPD of liquefaction per day

#   CAPEX - daily, for fast testing 
ELECTROLYSIS_CAPEX_DOL_kgPD = ELECTROLYSIS_CAPEX_DOL_TPD / 1000
COMPRESSOR_CAPEX_DOL_kgPD = COMPRESSOR_CAPEX_DOL_TPD / 1000
LIQUEFACTION_CAPEX_DOL_kgPD = LIQUEFACTION_CAPEX_DOL_TPD / 1000
FUELCELL_CAPEX_DOL_kgPD = FUELCELL_CAPEX_DOL_TPD / 1000

ELECTROLYSIS_CAPEX_DOL_kgPD = NUM_DAYS*ELECTROLYSIS_CAPEX_DOL_kgPD / (FACILITY_LIFETIME * 365)
COMPRESSOR_CAPEX_DOL_kgPD = NUM_DAYS*COMPRESSOR_CAPEX_DOL_kgPD / (FACILITY_LIFETIME * 365 )
LIQUEFACTION_CAPEX_DOL_kgPD = NUM_DAYS*LIQUEFACTION_CAPEX_DOL_kgPD / (FACILITY_LIFETIME * 365)
FUELCELL_CAPEX_DOL_kgPD = NUM_DAYS*FUELCELL_CAPEX_DOL_kgPD / (FACILITY_LIFETIME * 365)
print(LIQUEFACTION_CAPEX_DOL_kgPD)
print(ELECTROLYSIS_CAPEX_DOL_kgPD)
print(len(hourly_dayahead_dol_per_kwh))
print(hourly_dayahead_dol_per_kwh)

32.273972602739725
20.82191780821918
2400
0       0.02010
1       0.01962
2       0.01921
3       0.01879
4       0.02068
         ...   
2395    0.03264
2396    0.02714
2397    0.02412
2398    0.02154
2399    0.01596
Name: price_dayahead, Length: 2400, dtype: float64


In [28]:
problem, operations, facility = run_profit_maximisation(
        SOLAR_CAPACITY,
        SOLAR_PPA_DOL_KWH,
        hourly_solar_production_kwh,
        hourly_dayahead_dol_per_kwh,
        ELECTROLYSIS_EFFICIENCY,
        COMPRESSION_EFFICIENCY,
        LIQUEFACTION_EFFICIENCY,
        FUEL_CELL_EFFICIENCY,
        LIQUEFACTION_MAX_UP,
        LIQUEFACTION_MAX_DOWN,
        OFFTAKE_TPD,
        NUM_DAYS,
        ELECTROLYSIS_CAPEX_DOL_kgPD,
        COMPRESSOR_CAPEX_DOL_kgPD,
        LIQUEFACTION_CAPEX_DOL_kgPD,
        FUELCELL_CAPEX_DOL_kgPD,
        H2_SALES_PRICE_DOL_per_kg,
        GREEN_THRESHOLD_kg_per_kg,
        ERCTO_CO2_kg_per_kwh,
        T
        )
# OUTPUT
power_opex_total = np.sum(operations['hourly_dayahead_dol_per_kwh'] @ operations['realtime_production_kwh']) 
power_opex_total += np.sum(SOLAR_PPA_DOL_KWH * operations['solar_production_kwh'])
power_opex_per_kg = power_opex_total/sum(operations['liquefaction_produced_kg'])

capex_total = facility['electrolyser_capacity_ph'] * 24 *ELECTROLYSIS_CAPEX_DOL_kgPD + \
    facility['compressor_capacity_ph'] * 24 * COMPRESSOR_CAPEX_DOL_kgPD + \
   facility['liquefaction_capacity_ph'] * 24 * LIQUEFACTION_CAPEX_DOL_kgPD


capex_per_kg = capex_total / sum(operations['liquefaction_produced_kg'])
print(f"total offtake {sum(operations['liquefaction_produced_kg'])}")
print(f"power opex / kg {power_opex_per_kg}")
print(f"capex per kg {capex_per_kg}")
print(f"total dol per kg {power_opex_per_kg + capex_per_kg}")
print(f"profit {facility['total_profit']}")

print(f"capacity of electrolysis (tpd): {facility['electrolyser_capacity_ph']*24}, capex {facility['electrolyser_capacity_ph'] *24 *ELECTROLYSIS_CAPEX_DOL_kgPD}")
print(f"capacity of compressor (tpd) {facility['compressor_capacity_ph']*24}, capex {facility['compressor_capacity_ph'] * 24 * COMPRESSOR_CAPEX_DOL_kgPD}")
print(f"capacity of liquefaction (tpd) {facility['liquefaction_capacity_ph']*24}, capex {facility['liquefaction_capacity_ph'] * LIQUEFACTION_CAPEX_DOL_kgPD}")
print(f"capacity of fuelcell (tpd) {facility['fuelcell_capacity_ph']*24}, capex {facility['fuelcell_capacity_ph'] * FUELCELL_CAPEX_DOL_kgPD}")

(24,)
total offtake 4.49832510260607
power opex / kg 2.071584177170855
capex per kg 1.8030469964005673
total dol per kg 3.874631173571422
profit 0.3509086431175248
capacity of electrolysis (tpd): 11.828021903564887, capex 2.462820999098442
capacity of compressor (tpd) 11.828021903564887, capex 1.1665994206255779
capacity of liquefaction (tpd) 13.885093107449418, capex 0.18671963105679698
capacity of fuelcell (tpd) 19.44, capex 0.008876712328767125


In [51]:
for hydrogen_sales_price in [2.0,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]:
    problem, operations, facility = run_profit_maximisation(
        SOLAR_CAPACITY,
        SOLAR_PPA_DOL_KWH,
        hourly_solar_production_kwh,
        hourly_dayahead_dol_per_kwh,
        ELECTROLYSIS_EFFICIENCY,
        COMPRESSION_EFFICIENCY,
        LIQUEFACTION_EFFICIENCY,
        FUEL_CELL_EFFICIENCY,
        LIQUEFACTION_MAX_UP,
        LIQUEFACTION_MAX_DOWN,
        OFFTAKE_TPD,
        NUM_DAYS,
        ELECTROLYSIS_CAPEX_DOL_kgPD,
        COMPRESSOR_CAPEX_DOL_kgPD,
        LIQUEFACTION_CAPEX_DOL_kgPD,
        FUELCELL_CAPEX_DOL_kgPD,
        hydrogen_sales_price,
        GREEN_THRESHOLD_kg_per_kg,
        ERCTO_CO2_kg_per_kwh,
        T
        )

    operations.to_csv(os.path.join('data', 'results','deterministic',f'operations_results_{hydrogen_sales_price}.csv'))
    
    with open(os.path.join('data', 'results','deterministic',f'facility_summary_results_{hydrogen_sales_price}.json'), 'w') as file:
        json.dump(facility, file, indent=4)

    # print total OPEX
    # print(f"{hydrogen_sales_price} & {round(1000*facility['total_profit']*(365/NUM_DAYS),2)} & \
    #       {round(facility['h2_revenue']*(365/NUM_DAYS),2)} & {round(facility['wholesale_revenue']*(365/NUM_DAYS),2)} & \
    #       {round(facility['solar_cost']*(365/NUM_DAYS),2)} & {round(facility['wholesale_cost']*(365/NUM_DAYS),2)} \\\\")

    # print te
    print(f"{hydrogen_sales_price} & {round(1000*facility['total_profit']*(365/NUM_DAYS),2)} & {round(1000*facility['total_profit']*(FACILITY_LIFETIME* 365/NUM_DAYS),2)}\\\\")

2.0 & 0.0 & 0.0\\
2.5 & 0.0 & 0.0\\
3 & 0.0 & 0.0\\
3.5 & 0.0 & 0.0\\
4 & 993933.07 & 24848326.72\\
4.5 & 3797584.5 & 94939612.58\\
5 & 7382119.51 & 184552987.73\\
5.5 & 11396349.82 & 284908745.4\\
6 & 15649061.24 & 391226531.01\\
6.5 & 20035053.33 & 500876333.33\\
7 & 24513125.6 & 612828140.0\\


In [53]:
612828140.0 / ((2204 + 1044 + 1912)*25*1000)

4.7506057364341086